In [ ]:
# pip install langchain qdrant_client openai tiktoken streamlit

In [4]:
from dotenv import load_dotenv
import streamlit as st

In [5]:
# Imports
from langchain.vectorstores import qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import qdrant_client
import os


In [6]:
# create qdrant client
client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [10]:
# create collection

# define parameters for vectors
vectors_config = qdrant_client.models.VectorParams(
    size=1536, # vector size for OpenAI
    distance=qdrant_client.models.Distance.COSINE
)

# create collection
client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)

AttributeError: module 'qdrant_client' has no attribute 'models'

In [ ]:
# define get vector store

def get_vector_store():
    
    # creat qdrant client
    client = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )
    
    embeddings = OpenAIEmbeddings()

    # get collection name
    vector_store = Qdrant(
        client=client, 
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"), 
        embeddings=embeddings,
    )
    
    return vector_store

In [ ]:
# create main

def main():
    load_dotenv()
    
    st.set_page_config(page_title="Ask Qdrant")
    st.header("Ask your remote database 💬")
    
    # create vector store
    vector_store = get_vector_store()
    
    # create chain 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type="stuff",
        retriever=vector_store.as_retriever()
    )

    # show user input
    user_question = st.text_input("Ask a question about your PDF:")
    if user_question:
        st.write(f"Question: {user_question}")
        answer = qa.run(user_question)
        st.write(f"Answer: {answer}")
    
        
if __name__ == '__main__':
    main()